In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technicals.indicators import MACD
from simulation.guru_tester import GuruTester

In [3]:
BUY = 1
SELL = -1
NONE = 0

def apply_signal(row):
    if row.direction == BUY and row.mid_l > row.EMA_100:
        return BUY
    if row.direction == SELL and row.mid_h < row.EMA_100:
        return SELL
    return NONE        

In [4]:
pair = "EUR_USD"
df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [5]:
df_an = MACD(df_an)
df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [6]:
def apply_cross(row):
    if row.macd_delta > 0 and row.macd_delta_prev < 0:
        return BUY
    if row.macd_delta < 0 and row.macd_delta_prev > 0:
        return SELL
    return NONE 

In [7]:
df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
df_an['direction'] = df_an.apply(apply_cross, axis=1)

candle_dir = df_an.mid_c - df_an.mid_o
df_an['candle_dir'] = [BUY if x >= 0 else SELL for x in candle_dir]

In [8]:
df_an.direction.value_counts()

direction
 0    34259
 1     1477
-1     1476
Name: count, dtype: int64

In [9]:
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,ask_l,ask_c,MACD,SIGNAL,HIST,EMA_100,macd_delta,macd_delta_prev,direction,candle_dir
0,2017-01-09 02:00:00+00:00,368,1.05422,1.05424,1.05327,1.05327,1.05415,1.05417,1.05320,1.05320,...,1.05334,1.05334,-0.000922,-0.000610,-0.000312,1.052805,-0.000312,NaN,0,-1
1,2017-01-09 03:00:00+00:00,431,1.05324,1.05347,1.05264,1.05298,1.05318,1.05339,1.05257,1.05290,...,1.05271,1.05305,-0.000951,-0.000678,-0.000273,1.052809,-0.000273,-0.000312,0,-1
2,2017-01-09 04:00:00+00:00,282,1.05300,1.05300,1.05238,1.05266,1.05293,1.05293,1.05231,1.05259,...,1.05245,1.05272,-0.000988,-0.000740,-0.000248,1.052806,-0.000248,-0.000273,0,-1
3,2017-01-09 05:00:00+00:00,604,1.05264,1.05322,1.05166,1.05240,1.05257,1.05316,1.05157,1.05234,...,1.05174,1.05247,-0.001027,-0.000798,-0.000230,1.052796,-0.000230,-0.000248,0,-1
4,2017-01-09 06:00:00+00:00,509,1.05238,1.05366,1.05193,1.05327,1.05230,1.05360,1.05186,1.05321,...,1.05200,1.05333,-0.000977,-0.000834,-0.000143,1.052807,-0.000143,-0.000230,0,1


In [10]:
def run_pair(pair, time_d=1):
    df_an = pd.read_pickle(f"../data/{pair}_H{time_d}.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = MACD(df_an)
    df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
    
    df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
    df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
    df_an['direction'] = df_an.apply(apply_cross, axis=1)
    candle_dir = df_an.mid_c - df_an.mid_o
    df_an['candle_dir'] = [1 if x >= 0 else -1 for x in candle_dir]
    
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
            'direction', 'candle_dir', 'EMA_100']
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True,
        time_d = time_d
    )
    
    gt.run_test()
    return gt.df_results
    

In [11]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p, time_d=1)))

prepare_data...
run_test...
Result: -523.5
prepare_data...
run_test...
Result: -51.5
prepare_data...
run_test...
Result: -6.0
prepare_data...
run_test...
Result: -119.0


In [12]:
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -523.5
EUR_USD -51.5
GBP_JPY -6.0
USD_CAD -119.0


In [13]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p, time_d=4)))
for r in res:
    print(r['pair'], r['res'].result.sum())

prepare_data...
run_test...
Result: -65.0
prepare_data...
run_test...
Result: 10.5
prepare_data...
run_test...
Result: 35.5
prepare_data...
run_test...
Result: -23.0
AUD_NZD -65.0
EUR_USD 10.5
GBP_JPY 35.5
USD_CAD -23.0
